# Stabil párosítás

1. feladat: 5 végzős hallgató jelentkezik 5 cég 1-1 gyakornoki helyére. Az alábbi táblázat reprezentálja a hallgatók preferenciáit az egyes cégek által kínált gyakornoki pozíciókon:
$$
\begin{array}{l|ccccc}
 \text{Hallgató preferenciák} & \text{1. cég} & \text{2. cég} & \text{3. cég} & \text{4. cég} & \text{5. cég} \\ \hline
\text{1. hallgató} & 1 & 5 & 4 & 2 & 3 \\
\text{2. hallgató} & 2 & 4 & 1 & 3 & 5 \\
\text{3. hallgató} & 2 & 1 & 3 & 5 & 4 \\
\text{4. hallgató} & 5 & 3 & 2 & 4 & 1 \\
\text{5. hallgató} & 4 & 3 & 5 & 1 & 2
\end{array}
$$
azaz az 1. hallgató preferenciái a táblázat első sorában: leginkább az 1. céghez szeretne menni (1-es a preferencia listájában), míg legkevésbé a 2.céghez (5. a preferencia listájában.

A cégek szintén rendelkeznek a hallgatókra vonatkozó preferenciákkal:
$$
\begin{array}{l|ccccc}
 \text{Cég preferenciák} & \text{1. cég} & \text{2. cég} & \text{3. cég} & \text{4. cég} & \text{5. cég} \\ \hline
\text{1. hallgató} & 5 & 1 & 2 & 3 & 3 \\
\text{2. hallgató} & 2 & 4 & 5 & 4 & 1 \\
\text{3. hallgató} & 3 & 5 & 4 & 1 & 4 \\
\text{4. hallgató} & 1 & 3 & 3 & 2 & 5 \\
\text{5. hallgató} & 4 & 2 & 1 & 5 & 2
\end{array}
$$
azaz az 1. cég preferenciái a táblázat első oszlopában: leginkább a 4. hallgatót szeretné felvenni (1), és legkevésbé az 1. hallgatót (5).

* A hallgatók cégekhez való **hozzárendelése** meghatároz egy hallgató-cég párosítást: minden hallgató 1 gyakornoki pozíciót tölt be, minden cég gyakornoki pozícióját betölti 1 hallgató.
* Ez a párosítás akkor lesz **stabil**, ha nem létezik ún. *blokkoló hallgató-cég pár*
* Egy adott $(i,j)$ hallgató-cég pár akkor **blokkoló**, ha:
    * az $i$ hallgató preferálja a $j$ céget a párosítás által hozzá rendelt céghez képest
    * a $j$ cég preferálja az $i$ hallgatót a párosítás által hozzá rendelt hallgatóhoz képest

A stabil párosítás LP modelljében az **$x_{ij} \in \{0,1\}$ bináris döntési változók**: $x_{ij} = 1 \iff$ $i$ hallgató a $j$ céghez kerül.

Ekkor a stabil párosítás utóbbi része nagyjából magától értetődően érhető el:
* kiegyensúlyozott hozzárendelési feladat korlátaira lesz szükségünk
* azaz $\displaystyle \sum_{j = 1}^m x_{ij} = 1$ minden $i=1,\dots,m$ hallgató esetén
* és $\displaystyle \sum_{i = 1}^m x_{ij} = 1$ minden $j=1,\dots,m$ cég esetén

A stabilitás korlátai lesznek az egyik új elem:
* minden $(i,j)$ pár esetén az alábbiak legalább egyike kell, hogy teljesüljön
    * a párosításban $i$ hallgató a $j$ céghez kerül, azaz $x_{ij} = 1$
    * VAGY $i$ hallgató párja egy olyan $k$ cég ($x_{ik}=1$), melyet előrébb rangsorol, mint $j$-t: $k \succ_i j$
        * azaz $\displaystyle \sum_{k \succ_i j} x_{ik}=1$
    * VAGY $j$ cég párja egy olyan $l$ hallgató ($x_{lj}=1$), melyet előrébb rangsorol, mint $i$-t: $l \succ_j i$
        * azaz $\displaystyle \sum_{l \succ_j i} x_{lj}=1$
* a három közül legalább az egyik fennállása egy $(i,j)$ pár esetén elérhető a
$$
x_{ij} + \displaystyle \sum_{k \succ_i j} x_{ik} + \displaystyle \sum_{l \succ_j i} x_{lj} \geq 1
$$
korláttal, hiszen a 3 tag közül legalább az egyiknek így 1-nek kell lennie.

In [1]:
# egyes hallgatók preferenciái soronként (lényegében az első táblázat belseje):
P=[[1,5,4,2,3],
   [2,4,1,3,5],
   [2,1,3,5,4],
   [5,3,2,4,1],
   [4,3,5,1,2]]

# azon [i,k] párok, melyekre i hallgato preferálja k céget j céggel szemben:
# [i,k] for k in cegek if P[i-1][k-1] < P[i-1][j-1]

# egyes cégek preferenciái oszloponként (lényegében a második táblázat belseje):
Q=[[5,1,2,3,3],
   [2,4,5,4,1],
   [3,5,4,1,4],
   [1,3,3,2,5],
   [4,2,1,5,2]]

# azon [l,j] párok, melyekre j cég preferálja l hallgatót i hallgatóval szemben:
# [l,j] for l in hallgatok if Q[l-1][j-1] < Q[i-1][j-1]

Szabadon választhatunk még célfüggvényt!
* minimalizálhatjuk a stabil párosításban a hallgatók preferenciáit: hallgató-optimális SP
* minimalizálhatjuk a cégek preferenciáit: cég-optimális SP
* minimalizálhatjuk a preferenciák összegét: *kiegyensúlyozott* SP

In [4]:
# Megoldás: hallgató-optimális
from pulp import *

cegek = [1,2,3,4,5]
hallgatok = [1,2,3,4,5]
#ceg_csucsok = ['1','2','3','4','5']

stabil_par = pulp.LpProblem('stabil_parositas', LpMinimize)

x = LpVariable.dicts('x',[(i,j) for i in hallgatok for j in cegek], lowBound = 0, cat="Continuous")
# itt sincs szükség bináris változókra, elég az LP-relaxált egy csúcsponti optimális megoldását megtalálni

stabil_par += lpSum(P[i-1][j-1]*x[i,j] for i in hallgatok for j in cegek)

for i in hallgatok:
    stabil_par += lpSum(x[i,j] for j in cegek) == 1
    
for j in cegek:
    stabil_par += lpSum(x[i,j] for i in hallgatok) == 1
    
for i in hallgatok:
    for j in cegek:
        stabil_par += x[i,j] + lpSum(x[i,k] for k in cegek if P[i-1][k-1] < P[i-1][j-1]) + lpSum(x[l,j] for l in hallgatok if Q[l-1][j-1] < Q[i-1][j-1]) >= 1

megoldas = stabil_par.solve()

print(LpStatus[megoldas])
if LpStatus[megoldas] == 'Optimal':
    print('Stabil párosítás össz. preferenciái: z* = ', value(stabil_par.objective))
    for i in hallgatok:
        for j in cegek:
            print ("x"+str((i,j))+" =", x[i,j].varValue)

Optimal
Stabil párosítás össz. preferenciái: z* =  5.0
x(1, 1) = 1.0
x(1, 2) = 0.0
x(1, 3) = 0.0
x(1, 4) = 0.0
x(1, 5) = 0.0
x(2, 1) = 0.0
x(2, 2) = 0.0
x(2, 3) = 1.0
x(2, 4) = 0.0
x(2, 5) = 0.0
x(3, 1) = 0.0
x(3, 2) = 1.0
x(3, 3) = 0.0
x(3, 4) = 0.0
x(3, 5) = 0.0
x(4, 1) = 0.0
x(4, 2) = 0.0
x(4, 3) = 0.0
x(4, 4) = 0.0
x(4, 5) = 1.0
x(5, 1) = 0.0
x(5, 2) = 0.0
x(5, 3) = 0.0
x(5, 4) = 1.0
x(5, 5) = 0.0


In [3]:
# Megoldás: cég-optimális
cegek = [1,2,3,4,5]
hallgatok = [1,2,3,4,5]
#ceg_csucsok = ['1','2','3','4','5']

stabil_par = pulp.LpProblem('stabil_parositas', LpMinimize)

x = LpVariable.dicts('x',[(i,j) for i in hallgatok for j in cegek], lowBound = 0, cat="Continuous")
# itt sincs szükség bináris változókra, elég az LP-relaxált egy csúcsponti optimális megoldását megtalálni

stabil_par += lpSum(Q[i-1][j-1]*x[i,j] for i in hallgatok for j in cegek)

for i in hallgatok:
    stabil_par += lpSum(x[i,j] for j in cegek) == 1
    
for j in cegek:
    stabil_par += lpSum(x[i,j] for i in hallgatok) == 1
    
for i in hallgatok:
    for j in cegek:
        stabil_par += x[i,j] + lpSum(x[i,k] for k in cegek if P[i-1][k-1] < P[i-1][j-1]) + lpSum(x[l,j] for l in hallgatok if Q[l-1][j-1] < Q[i-1][j-1]) >= 1

megoldas = stabil_par.solve()

print(LpStatus[megoldas])
if LpStatus[megoldas] == 'Optimal':
    print('Stabil párosítás össz. preferenciái: z* = ', value(stabil_par.objective))
    for i in hallgatok:
        for j in cegek:
            print ("x"+str((i,j))+" =", x[i,j].varValue)

Optimal
Stabil párosítás össz. preferenciái: z* =  5.0
x(1, 1) = 0.0
x(1, 2) = 1.0
x(1, 3) = 0.0
x(1, 4) = 0.0
x(1, 5) = 0.0
x(2, 1) = 0.0
x(2, 2) = 0.0
x(2, 3) = 0.0
x(2, 4) = 0.0
x(2, 5) = 1.0
x(3, 1) = 0.0
x(3, 2) = 0.0
x(3, 3) = 0.0
x(3, 4) = 1.0
x(3, 5) = 0.0
x(4, 1) = 1.0
x(4, 2) = 0.0
x(4, 3) = 0.0
x(4, 4) = 0.0
x(4, 5) = 0.0
x(5, 1) = 0.0
x(5, 2) = 0.0
x(5, 3) = 1.0
x(5, 4) = 0.0
x(5, 5) = 0.0


In [4]:
# Megoldás: kiegyensúlyozott

cegek = [1,2,3,4,5]
hallgatok = [1,2,3,4,5]
#ceg_csucsok = ['1','2','3','4','5']

stabil_par = pulp.LpProblem('stabil_parositas', LpMinimize)

x = LpVariable.dicts('x',[(i,j) for i in hallgatok for j in cegek], lowBound = 0, cat="Continuous")
# itt sincs szükség bináris változókra, elég az LP-relaxált egy csúcsponti optimális megoldását megtalálni

stabil_par += lpSum((P[i-1][j-1] + Q[i-1][j-1])*x[i,j] for i in hallgatok for j in cegek)

for i in hallgatok:
    stabil_par += lpSum(x[i,j] for j in cegek) == 1
    
for j in cegek:
    stabil_par += lpSum(x[i,j] for i in hallgatok) == 1
    
for i in hallgatok:
    for j in cegek:
        stabil_par += x[i,j] + lpSum(x[i,k] for k in cegek if P[i-1][k-1] < P[i-1][j-1]) + lpSum(x[l,j] for l in hallgatok if Q[l-1][j-1] < Q[i-1][j-1]) >= 1

megoldas = stabil_par.solve()

print(LpStatus[megoldas])
if LpStatus[megoldas] == 'Optimal':
    print('Stabil párosítás össz. preferenciái: z* = ', value(stabil_par.objective))
    for i in hallgatok:
        for j in cegek:
            print ("x"+str((i,j))+" =", x[i,j].varValue)

Optimal
Stabil párosítás össz. preferenciái: z* =  24.0
x(1, 1) = 0.0
x(1, 2) = 0.0
x(1, 3) = 0.0
x(1, 4) = 1.0
x(1, 5) = 0.0
x(2, 1) = 1.0
x(2, 2) = 0.0
x(2, 3) = 0.0
x(2, 4) = 0.0
x(2, 5) = 0.0
x(3, 1) = 0.0
x(3, 2) = 1.0
x(3, 3) = 0.0
x(3, 4) = 0.0
x(3, 5) = 0.0
x(4, 1) = 0.0
x(4, 2) = 0.0
x(4, 3) = 1.0
x(4, 4) = 0.0
x(4, 5) = 0.0
x(5, 1) = 0.0
x(5, 2) = 0.0
x(5, 3) = 0.0
x(5, 4) = 0.0
x(5, 5) = 1.0


Térjünk vissza az eredeti feladatra. Tegyük fel, hogy az a cél, hogy a legrosszabbul járó hallgató is a lehető legjobban járjon. Keressünk ennek a célnak megfelelő stabil párosítást!

In [1]:
# Visszaállítjuk az eredeti véleménymátrixokat
P=[[1,5,4,2,3],
   [2,4,1,3,5],
   [2,1,3,5,4],
   [5,3,2,4,1],
   [4,3,5,1,2]]

Q=[[5,1,2,3,3],
   [2,4,5,4,1],
   [3,5,4,1,4],
   [1,3,3,2,5],
   [4,2,1,5,2]]

In [2]:
# Bevezetünk egy új változót, ami a felsp korlátja lesz az összes kiválasztott hallgatói preferencia-értéknek
# Ezt a változót fogjuk minimalizálni
# Ez garantálja, hogy a legrosszabbul járó hallgató által kiválasztott preferencia érték a lehető legkisebb
# Megjegyzés: természetesen a fentebbi hallgató-optimális megoldás alapján t=1 optimális célfüggvény értékre számítunk
from pulp import *

cegek = [1,2,3,4,5]
hallgatok = [1,2,3,4,5]

stabil_par = pulp.LpProblem('stabil_parositas', LpMinimize)

x = LpVariable.dicts('x',[(i,j) for i in hallgatok for j in cegek], lowBound = 0, cat="Continuous")
t = LpVariable('t', lowBound = 0, cat='Continuous')

stabil_par += t

for i in hallgatok:
    stabil_par += lpSum(x[i,j] for j in cegek) == 1
    
for j in cegek:
    stabil_par += lpSum(x[i,j] for i in hallgatok) == 1
    
for i in hallgatok:
    for j in cegek:
        stabil_par += x[i,j] + lpSum(x[i,k] for k in cegek if P[i-1][k-1] < P[i-1][j-1]) + lpSum(x[l,j] for l in hallgatok if Q[l-1][j-1] < Q[i-1][j-1]) >= 1

for i in hallgatok:
    for j in cegek:
        stabil_par += P[i-1][j-1]*x[i,j] <= t

megoldas = stabil_par.solve()

print(LpStatus[megoldas])
if LpStatus[megoldas] == 'Optimal':
    print('Stabil párosítás össz. preferenciái: z* = ', value(stabil_par.objective))
    for i in hallgatok:
        for j in cegek:
            print ("x"+str((i,j))+" =", x[i,j].varValue)

Optimal
Stabil párosítás össz. preferenciái: z* =  0.83333333
x(1, 1) = 0.83333333
x(1, 2) = 0.16666667
x(1, 3) = 0.0
x(1, 4) = 0.0
x(1, 5) = 0.0
x(2, 1) = 0.0
x(2, 2) = 0.0
x(2, 3) = 0.83333333
x(2, 4) = 0.0
x(2, 5) = 0.16666667
x(3, 1) = 0.0
x(3, 2) = 0.83333333
x(3, 3) = 0.0
x(3, 4) = 0.16666667
x(3, 5) = 0.0
x(4, 1) = 0.16666667
x(4, 2) = 0.0
x(4, 3) = 0.0
x(4, 4) = 0.0
x(4, 5) = 0.83333333
x(5, 1) = 0.0
x(5, 2) = 0.0
x(5, 3) = 0.16666667
x(5, 4) = 0.83333333
x(5, 5) = 0.0


Az újonnan hozzáadott P[i-1][j-1]*x[i,j] <= t korlátok már tönkreteszik a speciális struktúrát:
<br>nem lesznek a megengedett megoldáshalmaz sarokpontjai feltétlenül egész értékűek
<br>és ezáltal nem lesz garantált az egész értékű optimális megoldás 'automatikus' létezése nem-negatív változók esetén
<br>szükség van bináris változók explicit előírására!